In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_olivetti_faces

- Narysować wybrane obrazy.

- Wykorzystać algorytm PCA do kompresji zbioru Olivetti Faces. Dobrać odpowiednią liczbę składowych. Po transformacji obliczyć stopień kompresji. Rozmiar obrazka: liczba wartości numerycznych

- Przeprowadzić transformację odwrotną (inverse_transform). Narysować, porównać z pkt. 0. Obliczyć błąd rekonstrukcji w postaci błędu RMSE dla każdego obrazu.

- Przygotować kilka / kilkanaście zmodyfikowanych obrazów (np. obróconych o 90 stopni, przyciemnionych, odbitych w poziomie).

- Korzystając z modelu wyuczonego w pkt. 1 przeprowadzić transformację, a następnie odwrotną transformację obrazów z pkt. 3.

- Obliczyć błąd rekonstrukji dla każdego typu modyfikacji. Porównać z wartościami błędu uzyskanymi w pkt. 2.

- Czy PCA może służyć do wykrywania pewnego typu anomalii w zdjęciach twarzy? Jeżeli tak to jakich?

In [ ]:
faces = fetch_olivetti_faces()
data = faces['data']
images = faces['images']
test_images = images[[1, 50, 100, 200, 300],:,:]

# Rysowanie obrazów

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    axs[i].imshow(test_images[i])

# Kompresja

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=400)

data_pca = pca.fit_transform(data)

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.xlim(0, 200)
plt.ylabel('cumulative explained variance')

Przy około 100 komponentach wyjaśnionych jest 90% warinacji.

In [ ]:
pca = PCA(n_components=100)
data_pca2 = pca.fit_transform(data)

In [ ]:
data.shape

In [ ]:
data_pca2.shape

In [ ]:
print(f"Stopień kompresji to {(data.shape[0] * data.shape[1]) / (data_pca2.shape[0] * data_pca2.shape[1])}")

# Transformacja odwrotna

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
inv_t = pca.inverse_transform(data_pca2)
idx = [1, 50, 100, 200, 300]
for i in range(5):
    axs[i].imshow(inv_t[idx[i]].reshape(64, 64))

Różnica po inverse transforamtion jest niezauważalna.

In [ ]:
from sklearn.metrics import mean_squared_error as rmse

In [ ]:
rmse_arr = []

for i in range(400):
    rmse_arr.append(rmse(data[i], inv_t[i], squared = True))

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(range(1, len(rmse_arr)+1), rmse_arr)
plt.xlabel('picture')
plt.xlim(0, 400)
plt.ylabel('rmse')

In [ ]:
np.mean(rmse_arr)

Błąd rekonstrukcji jest bardzo niski, wynosi troche ponad 0.001.

# Modyfikacja obrazów
## Obrót

In [ ]:
from skimage.transform import rotate

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
rotated_images = []
for i in range(5):
    rotated_images.append(rotate(test_images[i], -90))
    axs[i].imshow(rotated_images[i])

## Obrót po PCA i inverse trasformation

In [ ]:
rotated_images_pca = pca.inverse_transform(pca.transform(np.array(rotated_images).reshape(5, -1)))

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    axs[i].imshow(rotated_images_pca[i].reshape(64,64))

## Przyciemnienie

In [ ]:
from skimage import exposure

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
darker_images = []
for i in range(5):
    darker_images.append(exposure.adjust_gamma(test_images[i], 5))
    axs[i].imshow(darker_images[i])

## Przyciemnienie po PCA i inverse trasformation

In [ ]:
darker_images_pca = pca.inverse_transform(pca.transform(np.array(darker_images).reshape(5, -1)))

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    axs[i].imshow(darker_images_pca[i].reshape(64,64))

## Odbicie w poziomie

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
fliped_images = []
for i in range(5):
    fliped_images.append(test_images[i][:,::-1])
    axs[i].imshow(fliped_images[i])

## Odbicie w poziomie po PCA i inverse trasformation

In [ ]:
fliped_images_pca = pca.inverse_transform(pca.transform(np.array(fliped_images).reshape(5, -1)))

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    axs[i].imshow(fliped_images_pca[i].reshape(64,64))

# Błąd transformacji

In [ ]:
# błąd dla wszystkich obrazów przy obrocie
rotated_all = np.array([rotate(images[i], -90) for i in range(len(images))]).reshape(len(images), -1)
rotated_all_pca = pca.inverse_transform(
    pca.transform(
        rotated_all
    )
)

rmse_rotated = np.array([rmse(rotated_all[i], rotated_all_pca[i], squared = True) for i in range(len(rotated_all))])

# błąd dla wszystkich obrazów po przciemnenieu
darker_all = np.array([exposure.adjust_gamma(images[i], 5) for i in range(len(images))]).reshape(len(images), -1)
darker_all_pca = pca.inverse_transform(
    pca.transform(
        darker_all
    )
)

rmse_darker = np.array([rmse(darker_all[i], darker_all_pca[i], squared = True) for i in range(len(darker_all_pca))])

# błąd dla wszystkich obrazów po odbiciu
flipped_all = np.array([images[i][:,::-1] for i in range(len(images))]).reshape(len(images), -1)
flipped_all_pca = pca.inverse_transform(
    pca.transform(
        flipped_all
    )
)

rmse_flipped = np.array([rmse(flipped_all[i], flipped_all_pca[i], squared = True) for i in range(len(flipped_all))])


In [ ]:
df = pd.DataFrame({
    'transformation' : ['normal', 'rotation', 'dark', 'flip'],
    'rmse' : [np.mean(rmse_arr), np.mean(rmse_rotated), np.mean(rmse_darker), np.mean(rmse_flipped)]
    })

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,6))

sns.barplot(data = df, x = 'transformation', y = 'rmse',
            color = 'dodgerblue',
            order = df.sort_values('rmse')['transformation'], ax = ax)
ax.set_title('RMSE for transformations')
ax.set_xlabel('')

Największe RMSE jest dla obrócenia zdjęcia o 90, można było to wywnioskować już po narysowanych obrazach, po transformacji i transformacji odwrotenj obrazy w ogóle nie były obrócone i prawie wcale nie są do siebie podobne. 
Przyciemnione obrazy również straciły na jakości jednak wciąż można je rozpoznać a efekt przyciemnienia się zachował.
Na obazach odbitych w poziomie widać podobieństow i zachowanie efektu.

PCA może być wykorzytywane w wykrywaniu anomalii w zdjęciach, szczególnie w obrotu (można na przykład stworzyć narzędzie do automatycznego obracania zdjęć w dobrą stronę).